# Chronos-2 Hyperparameter Tuning con Optuna
Questo notebook esegue il tuning degli iperparametri con Optuna per il modello Chronos-2,
sia per il modello generale sia per i modelli per categoria. Verranno salvati i top-3 risultati per modello.

In [1]:
import sys
import os
from pathlib import Path
import json
import logging

ROOT = Path('..').resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(ROOT / 'model') not in sys.path:
    sys.path.insert(0, str(ROOT / 'model'))

import optuna
import pandas as pd
import numpy as np
import torch

from chronos import Chronos2Pipeline
from core.data import GICS_LEVEL_1, create_multivariate_windows, prepare_data_for_chronos
from core.eval import evaluate_model_on_test
from tiingo_data.download_data import get_daily_returns_data_cached
from utils import create_train_val_split, get_device

logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger()

## Caricamento dati e impostazioni principali
Carichiamo i ritorni giornalieri e definiamo parametri base (context/prediction length, stride, ecc.).

# Chronos-2 Hyperparameter Tuning con Optuna
Questo notebook esegue il tuning degli iperparametri con Optuna per il modello Chronos-2
sia sul dataset completo sia per categoria (settori GICS), salvando i risultati su disco.

In [2]:
import sys
import os
from pathlib import Path

current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

import pandas as pd
from tiingo_data.download_data import get_daily_returns_data_cached
from core.tuning import tune_hyperparams_by_group_optuna


## Caricamento dati
Carichiamo i dati di ritorni giornalieri e definiamo la search space per Optuna.
Il batch size viene esplorato fino a 100.

In [ ]:
df_all = get_daily_returns_data_cached()
print(f"Dataset shape: {df_all.shape}")

search_space = {
    "context_length": {"type": "int", "low": 64, "high": 256, "step": 32},
    "learning_rate": {"type": "float", "low": 1e-6, "high": 5e-5, "log": True},
    "num_steps": {"type": "int", "low": 200, "high": 1500, "step": 100},
    "batch_size": {"type": "int", "low": 8, "high": 100, "step": 8},
    "stride": {"type": "int", "low": 25, "high": 100, "step": 25},
    "n_eval_samples": {"type": "int", "low": 50, "high": 200, "step": 50},
}

Dataset shape: (3997, 121)


## Esecuzione tuning con Optuna (globale + per categoria)
Esegue il tuning per il dataset completo e per ogni settore.
L'output include il best result e la lista completa dei risultati per gruppo.

In [ ]:
results_by_group = tune_hyperparams_by_group_optuna(
    df=df_all,
    search_space=search_space,
    test_size=1200,
    val_ratio=0.1,
    output_dir="outputs/chronos2_tuning2",
    cleanup_output_dir=True,
    n_trials=20,
)

[I 2026-01-10 00:31:34,832] A new study created in memory with name: no-name-e4eae107-b9c2-4818-8512-05922ce10e36
c:\Users\rosar\anaconda3\envs\chronos_dnlp\lib\site-packages\optuna\distributions.py:684: UserWarning: The distribution is specified by [8, 100] and step=8, but the range is not divisible by `step`. It will be replaced with [8, 96].
  warnings.warn(
c:\Users\rosar\anaconda3\envs\chronos_dnlp\lib\site-packages\transformers\modeling_utils.py:6105: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\bld\libtorch_1764032314068\work\c10/cuda/CUDAAllocatorConfig.h:35.)
  _ = torch.empty(byte_count // factor, dtype=torch.float16, device=device, requires_grad=False)
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
100,2.034100,2.456244
200,2.117700,2.492929
300,1.954400,2.507076
400,1.807500,2.522353
500,1.688500,2.643570
600,1.599500,2.688911
700,1.500400,2.770963
800,1.387500,2.922579
900,1.281300,2.923522
1000,1.164400,3.038292


INFO:chronos.chronos2.pipeline:Finetuned model saved to outputs\chronos2_tuning\global_trial_1\finetuned-ckpt
[I 2026-01-10 00:34:54,722] Trial 0 finished with value: 0.007350990083068609 and parameters: {'context_length': 160, 'prediction_length': 4, 'learning_rate': 3.1684008576786285e-05, 'num_steps': 1400, 'batch_size': 16, 'stride': 25, 'n_eval_samples': 100}. Best is trial 0 with value: 0.007350990083068609.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
100,1.014100,0.872492
200,0.956200,0.922239
300,0.817200,0.832346
400,0.688000,0.889808
500,0.600400,0.953220
600,0.489100,1.012535
700,0.445300,1.039969
800,0.392400,1.069729
900,0.354700,1.070904
1000,0.344700,1.088015


INFO:chronos.chronos2.pipeline:Finetuned model saved to outputs\chronos2_tuning\global_trial_2\finetuned-ckpt
[I 2026-01-10 00:37:58,151] Trial 1 finished with value: 0.006212410517036915 and parameters: {'context_length': 192, 'prediction_length': 2, 'learning_rate': 2.2079598990791412e-05, 'num_steps': 1000, 'batch_size': 40, 'stride': 50, 'n_eval_samples': 50}. Best is trial 1 with value: 0.006212410517036915.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


...

INFO:chronos.chronos2.pipeline:Finetuned model saved to outputs\chronos2_tuning\Real Estate_trial_18\finetuned-ckpt
[I 2026-01-10 07:48:27,916] Trial 17 finished with value: 0.007792056538164616 and parameters: {'context_length': 224, 'prediction_length': 3, 'learning_rate': 1.8697747317188706e-05, 'num_steps': 1300, 'batch_size': 8, 'stride': 50, 'n_eval_samples': 150}. Best is trial 10 with value: 0.0054474701173603535.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
100,0.548700,2.575773
200,0.181100,2.586951
300,0.137100,2.663939
400,0.114700,2.701248
500,0.099800,2.725149
600,0.090200,2.712902
700,0.082400,2.708953
800,0.076100,2.752466
900,0.069900,2.737180


INFO:chronos.chronos2.pipeline:Finetuned model saved to outputs\chronos2_tuning\Real Estate_trial_19\finetuned-ckpt
[I 2026-01-10 07:50:07,671] Trial 18 finished with value: 0.006425850559026003 and parameters: {'context_length': 128, 'prediction_length': 4, 'learning_rate': 4.980371886941175e-05, 'num_steps': 900, 'batch_size': 64, 'stride': 75, 'n_eval_samples': 100}. Best is trial 10 with value: 0.0054474701173603535.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
100,0.692700,1.515561
200,0.217900,1.421752
300,0.158300,1.576528
400,0.123300,1.431479
500,0.107000,1.460695
600,0.096600,1.447630
700,0.087500,1.433613
800,0.080000,1.424811
900,0.073400,1.430165
1000,0.071700,1.415278


INFO:chronos.chronos2.pipeline:Finetuned model saved to outputs\chronos2_tuning\Real Estate_trial_20\finetuned-ckpt
[I 2026-01-10 07:52:45,906] Trial 19 finished with value: 0.006495630834251642 and parameters: {'context_length': 192, 'prediction_length': 2, 'learning_rate': 8.46309573491826e-06, 'num_steps': 1500, 'batch_size': 24, 'stride': 50, 'n_eval_samples': 200}. Best is trial 10 with value: 0.0054474701173603535.


## Salvataggio risultati
Salviamo i risultati completi e i best per ogni gruppo in formato CSV/JSON.

In [5]:
output_path = Path("outputs/tuning_results")
output_path.mkdir(parents=True, exist_ok=True)

records = []
best_records = []

for group_name, (best_result, all_results) in results_by_group.items():
    best_records.append({
        "group": group_name,
        **best_result.config,
        "mean_quantile_loss": best_result.mean_quantile_loss,
        "mean_mse": best_result.mean_mse,
        "mean_mae": best_result.mean_mae,
    })

    for result in all_results:
        records.append({
            "group": group_name,
            **result.config,
            "mean_quantile_loss": result.mean_quantile_loss,
            "mean_mse": result.mean_mse,
            "mean_mae": result.mean_mae,
        })

df_all_results = pd.DataFrame(records)
df_best_results = pd.DataFrame(best_records)

df_all_results.to_csv(output_path / "tuning_all_results.csv", index=False)
df_best_results.to_csv(output_path / "tuning_best_results.csv", index=False)

df_all_results.to_json(output_path / "tuning_all_results.json", orient="records", indent=2)
df_best_results.to_json(output_path / "tuning_best_results.json", orient="records", indent=2)

print("Saved results to", output_path)

Saved results to outputs\tuning_results


## Anteprima risultati

In [6]:
df_best_results.sort_values("mean_quantile_loss").head()

,group,context_length,prediction_length,learning_rate,num_steps,batch_size,stride,n_eval_samples,group_name,mean_quantile_loss,mean_mse,mean_mae
5,Consumer Staples,256,1,0.000046,600,96,75,100,Consumer Staples,0.004739,0.000210,0.010194
2,Health Care,192,2,0.000018,600,56,50,50,Health Care,0.005042,0.000251,0.011809
8,Communication Services,128,5,0.000022,1500,32,75,50,Communication Services,0.005098,0.000244,0.011434
10,Real Estate,256,4,0.000045,1500,8,75,150,Real Estate,0.005447,0.000279,0.012291
6,Industrials,192,2,0.000009,1300,72,100,200,Industrials,0.005620,0.000309,0.012660
